In [1]:
import pandas as pd
import numpy as np
from numba import njit, jit


In [70]:
qqq_df = pd.read_csv('QQQ_1D.csv')

In [21]:
# Translating the EMA functionality from Pine Script code (Trading View) into Python code
# to get the same result we see on Trading view
# data_src : numpy.ndarray
# returns : numpy.ndarray
@njit
def pine_ema(data_src, length):
    indicator = np.empty(shape = data_src.shape, dtype= data_src.dtype)
    smoothing_factor = 2
    alpha = smoothing_factor / (length + 1)
    

    for i in range(len(data_src)):
        # Edge cases
        if i == 0:
            indicator[i] = data_src[i]
            continue
        if i > 0 and np.isnan(indicator[i-1]):
            indicator[i] = data_src[i]
            continue
        if np.isnan(data_src[i]):
            indicator[i] = data_src[i]
            continue
        
        # EMA formula 
        indicator[i] = ( alpha * data_src[i] ) + ( 1 - alpha ) * indicator[i-1]

    indicator[ :length-1] = np.nan
    return indicator

pine_ema(qqq_df['close'].to_numpy(), 15)

array([         nan,          nan,          nan, ..., 429.84409011,
       429.16982885, 429.39860024])

In [10]:
%%timeit 
pine_ema(qqq_df.close.to_numpy(), 200)

6.2 µs ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [71]:
qqq_df['EMA_50'] = pine_ema(qqq_df.close.to_numpy(), 50)
len(qqq_df.index)
qqq_df.columns

Index(['time', 'open', 'high', 'low', 'close', 'Volume', 'EMA_50'], dtype='object')

In [72]:
## Appending a new row to the existing DF
new_data_df = pd.read_csv('QQQ_1D_f9c6d.csv')
qqq_df.loc[len(qqq_df.index)] = [new_data_df.iloc[-1].time,
                                 new_data_df.iloc[-1].open,
                                 new_data_df.iloc[-1].high,
                                 new_data_df.iloc[-1].low,
                                 new_data_df.iloc[-1].close,
                                 new_data_df.iloc[-1].Volume,
                                 np.nan]
qqq_df

,time,open,high,low,close,Volume,EMA_50
0,2019-12-02,205.11,205.180,201.7800,203.00,28121020,NaN
1,2019-12-03,200.32,201.490,199.2300,201.41,31425404,NaN
2,2019-12-04,202.49,202.910,202.1400,202.43,15336754,NaN
3,2019-12-05,203.15,203.150,201.8400,202.83,15038785,NaN
4,2019-12-06,204.25,205.145,204.1501,205.00,16562247,NaN
...,...,...,...,...,...,...,...
1104,2024-04-23,420.77,426.280,418.8715,425.07,44384741,432.285886
1105,2024-04-24,428.20,429.730,424.2000,426.51,48405226,432.059381
1106,2024-04-25,419.24,425.320,418.1400,424.45,57360968,431.760973
1107,2024-04-26,427.62,432.550,426.9200,431.00,41859388,431.731131
